In [1]:
## Prepare the summary stats about eQTL effect sizes used in Decision Tee analysis

# Libraries

In [21]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/scgrn_R_4_1//lib/R/library"


In [22]:
library(stringi)

# Parameters

In [23]:
# Path to the expression data

In [24]:
expression_data = '/lustre/groups/epigenereg01/workspace/projects/grn_dev_groningen/1K1K/expression_data/'

In [ ]:
# Path to save results to

In [25]:
results_path_analysis = '/lustre/groups/epigenereg01/workspace/projects/grn_dev_groningen/coeqtl_mapping/co_qtls_sceqtlgen/analysis/'

In [26]:
co_qtl_mapping_results = '/lustre/groups/epigenereg01/workspace/projects/grn_dev_groningen/coeqtl_mapping/co_qtls_sceqtlgen/'

In [27]:
output_path = '/lustre/groups/epigenereg01/workspace/projects/grn_dev_groningen/coeqtl_mapping/co_qtls_sceqtlgen/analysis/'

In [28]:
dataset_name = 'CD4_T.Qced.Normalized.SCs.Rds'

# Functions

In [29]:
## Function to load dataset
get_dataset = function(path){
    data = readRDS(path)
    }

# Data

## Load the co-eQTL testing result 1K1K

In [46]:
### Mapping needs to be done on a gene-pair + SNP level

In [49]:
gene_pairs = read.csv(paste0(results_path_analysis, 'F3_1K1K_decision_tree_input.csv'))

In [51]:
gene_pairs = unique(gene_pairs[,c('feature_id', 'snp_id')])

In [52]:
#is.na(str_extract(colnames(gene_pairs), 'Unnamed'))

In [53]:
gene_pairs = gene_pairs[,is.na(str_extract(colnames(gene_pairs), 'Unnamed'))]

In [54]:
gene_pairs = setDT(gene_pairs)

In [55]:
head(gene_pairs,2)

feature_id,snp_id
<chr>,<chr>
A1BG-AS1_AASS,19:58355068:G:A
A1BG-AS1_AASS,7:122023523:T:TTA


In [65]:
gene_pairs$gene1 = str_replace(gene_pairs$feature_id, '_.*', '')
gene_pairs$gene2 = str_replace(gene_pairs$feature_id, '.*_', '')

In [66]:
nrow(gene_pairs)

[1] 31497753

In [67]:
length(unique(gene_pairs$feature_id))

[1] 8648490

In [68]:
head(gene_pairs,2)

feature_id,snp_id,gene1,gene2
<chr>,<chr>,<chr>,<chr>
A1BG-AS1_AASS,19:58355068:G:A,A1BG-AS1,AASS
A1BG-AS1_AASS,7:122023523:T:TTA,A1BG-AS1,AASS


## Load the cis-eQTL information

In [69]:
### Provided by Marc-Jan

In [70]:
effect_sizes = read.csv(paste0(co_qtl_mapping_results , 'sc_eQTLGen.txt'), sep = '\t')

In [71]:
head(effect_sizes,2)

,snp_id,feature_id,beta,p_value,assessed_allele,QTL
,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,19:1451255:C:T,RPS15,-0.0333488,0.0004937455,C,19:1451255:C:T:RPS15
2,19:5524450:G:T,RPL36,0.0404422,0.0012431399,G,19:5524450:G:T:RPL36


In [85]:
### SNP gene mapping provided by Dan

In [86]:
snp_gene_mapping = read.csv(paste0(co_qtl_mapping_results , 'gene_snp_ref.tsv'), sep = '\t')

In [87]:
head(snp_gene_mapping,2)

,gene,snp
,<chr>,<chr>
1,A1BG-AS1,19:58355068:G:A
2,A1BG,19:58355068:G:A


# Map the effect sizes

In [76]:
### for gene 1

In [73]:
gene_pairs = merge(gene_pairs, effect_sizes, by.x = c('snp_id', 'gene1' ), by.y = c('snp_id', 'feature_id'), all.x = TRUE)

In [74]:
head(gene_pairs,2)

snp_id,gene1,feature_id,gene2,beta,p_value,assessed_allele,QTL
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
10:100055816:G:A,AAK1,AAK1_CWF19L1,CWF19L1,NA,NA,NA,NA
10:100055816:G:A,ABCC1,ABCC1_CWF19L1,CWF19L1,NA,NA,NA,NA


In [75]:
nrow(gene_pairs)

[1] 31497753

In [77]:
## for gene 2

In [78]:
gene_pairs = merge(gene_pairs, effect_sizes, by.x = c('snp_id', 'gene2' ), by.y = c('snp_id', 'feature_id'), all.x = TRUE)

In [79]:
nrow(gene_pairs)

[1] 31497753

In [80]:
head(gene_pairs,2)

snp_id,gene2,gene1,feature_id,beta.x,p_value.x,assessed_allele.x,QTL.x,beta.y,p_value.y,assessed_allele.y,QTL.y
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
10:100055816:G:A,CWF19L1,AAK1,AAK1_CWF19L1,NA,NA,NA,NA,-0.2145484,1.735811e-05,G,10:100055816:G:A:CWF19L1
10:100055816:G:A,CWF19L1,ABCC1,ABCC1_CWF19L1,NA,NA,NA,NA,-0.2145484,1.735811e-05,G,10:100055816:G:A:CWF19L1


In [81]:
## Check whether always a unique assignment

In [82]:
nrow(gene_pairs[!is.na(gene_pairs$beta.y),])

[1] 14757295

In [83]:
nrow(gene_pairs[!is.na(gene_pairs$beta.x),])   # for some twiche assignment

[1] 16740855

In [95]:
nrow(gene_pairs[!is.na(gene_pairs$beta.y) & !(is.na(gene_pairs$beta.x)),])  # 397 that are not uniquely identifyable

[1] 397

In [84]:
head(gene_pairs[!is.na(gene_pairs$beta.y) & !(is.na(gene_pairs$beta.x)),])

snp_id,gene2,gene1,feature_id,beta.x,p_value.x,assessed_allele.x,QTL.x,beta.y,p_value.y,assessed_allele.y,QTL.y
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
10:103845346:GAAA:G,SLK,SH3PXD2A,SH3PXD2A_SLK,0.19483690,4.633129e-08,GAAA,10:103845346:GAAA:G:SH3PXD2A,0.1114048,1.608851e-04,GAAA,10:103845346:GAAA:G:SLK
10:1045255:G:A,WDR37,IDI1,IDI1_WDR37,0.32364725,2.230188e-22,G,10:1045255:G:A:IDI1,0.2314044,4.802828e-10,G,10:1045255:G:A:WDR37
10:109904837:A:G,XPNPEP1,ADD3,ADD3_XPNPEP1,-0.06790962,2.729124e-04,A,10:109904837:A:G:ADD3,-0.2175021,3.983823e-09,A,10:109904837:A:G:XPNPEP1
10:58391266:A:G,UBE2D1,TFAM,TFAM_UBE2D1,0.11906039,9.386902e-07,A,10:58391266:A:G:TFAM,0.1198520,5.973472e-07,A,10:58391266:A:G:UBE2D1
10:62212113:A:T,RTKN2,ARID5B,ARID5B_RTKN2,0.19783812,3.931072e-13,A,10:62212113:A:T:ARID5B,0.4110509,8.796773e-36,A,10:62212113:A:T:RTKN2
10:70188453:A:AC,TYSND1,SAR1A,SAR1A_TYSND1,-0.23179707,6.831258e-09,A,10:70188453:A:AC:SAR1A,0.1858658,1.062237e-08,A,10:70188453:A:AC:TYSND1


In [88]:
## Mapp the egene information based on mapping by Dan

In [89]:
snp_gene_mapping$eGene = 1

In [90]:
gene_pairs = merge(gene_pairs, snp_gene_mapping, by.x = c('snp_id', 'gene1'), by.y = c('snp', 'gene'), all.x  = TRUE)  # for gene 1
gene_pairs = merge(gene_pairs, snp_gene_mapping, by.x = c('snp_id', 'gene2'), by.y = c('snp', 'gene'), all.x  = TRUE)  # for gene 2

In [91]:
head(gene_pairs,2)

snp_id,gene2,gene1,feature_id,beta.x,p_value.x,assessed_allele.x,QTL.x,beta.y,p_value.y,assessed_allele.y,QTL.y,eGene.x,eGene.y
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
10:100055816:G:A,CWF19L1,AAK1,AAK1_CWF19L1,NA,NA,NA,NA,-0.2145484,1.735811e-05,G,10:100055816:G:A:CWF19L1,NA,1
10:100055816:G:A,CWF19L1,ABCC1,ABCC1_CWF19L1,NA,NA,NA,NA,-0.2145484,1.735811e-05,G,10:100055816:G:A:CWF19L1,NA,1


In [92]:
nrow(gene_pairs)

[1] 31497753

In [94]:
## Check whether unique assignment
nrow(gene_pairs[!is.na(gene_pairs$eGene.x),])
nrow(gene_pairs[!is.na(gene_pairs$eGene.y),])  ## same as with direct co-eQTL mapping

[1] 16740855

[1] 14757295

In [105]:
## Determine co-eQTL effect size for the gene-pair
# in case of two potential egenes take sum for now (alternative mean?)

In [106]:
gene_pairs$QTL_beta = 0
gene_pairs$QTL_beta[(!is.na(gene_pairs$beta.x)) & (is.na(gene_pairs$beta.y))] = gene_pairs$beta.x[(!is.na(gene_pairs$beta.x)) & (is.na(gene_pairs$beta.y))]
gene_pairs$QTL_beta[(is.na(gene_pairs$beta.x)) & (!is.na(gene_pairs$beta.y))] = gene_pairs$beta.y[(is.na(gene_pairs$beta.x)) & (!is.na(gene_pairs$beta.y))]
gene_pairs$QTL_beta[(!is.na(gene_pairs$beta.x)) & (!is.na(gene_pairs$beta.y))] = gene_pairs$beta.y[(!is.na(gene_pairs$beta.x)) & (!is.na(gene_pairs$beta.y))] +  gene_pairs$beta.x[(!is.na(gene_pairs$beta.x)) & (!is.na(gene_pairs$beta.y))]

In [107]:
nrow(gene_pairs)

[1] 31497753

In [108]:
nrow(gene_pairs[gene_pairs$QTL_beta == 0,] ) ## check mapping for all

[1] 0

In [109]:
head(gene_pairs,2)

snp_id,gene2,gene1,feature_id,beta.x,p_value.x,assessed_allele.x,QTL.x,beta.y,p_value.y,assessed_allele.y,QTL.y,eGene.x,eGene.y,QTL_beta
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
10:100055816:G:A,CWF19L1,AAK1,AAK1_CWF19L1,NA,NA,NA,NA,-0.2145484,1.735811e-05,G,10:100055816:G:A:CWF19L1,NA,1,-0.2145484
10:100055816:G:A,CWF19L1,ABCC1,ABCC1_CWF19L1,NA,NA,NA,NA,-0.2145484,1.735811e-05,G,10:100055816:G:A:CWF19L1,NA,1,-0.2145484


In [110]:
## Add co-eQTL p-value

In [111]:
gene_pairs$QTL_p_value = 0
gene_pairs$QTL_p_value[(!is.na(gene_pairs$p_value.x)) & (is.na(gene_pairs$p_value.y))] = gene_pairs$p_value.x[(!is.na(gene_pairs$p_value.x)) & (is.na(gene_pairs$p_value.y))]
gene_pairs$QTL_p_value[(is.na(gene_pairs$p_value.x)) & (!is.na(gene_pairs$p_value.y))] = gene_pairs$p_value.y[(is.na(gene_pairs$p_value.x)) & (!is.na(gene_pairs$p_value.y))]
gene_pairs$QTL_p_value[(!is.na(gene_pairs$p_value.x)) & (!is.na(gene_pairs$p_value.y))] = gene_pairs$p_value.y[(!is.na(gene_pairs$p_value.x)) & (!is.na(gene_pairs$p_value.y))] +  gene_pairs$p_value.x[(!is.na(gene_pairs$p_value.x)) & (!is.na(gene_pairs$p_value.y))]

In [114]:
## Define eGene

In [115]:
gene_pairs$eGene = ''
gene_pairs$eGene[(!is.na(gene_pairs$eGene.x)) & (is.na(gene_pairs$eGene.y))] = gene_pairs$gene1[(!is.na(gene_pairs$eGene.x)) & (is.na(gene_pairs$eGene.y))]
gene_pairs$eGene[(is.na(gene_pairs$eGene.x)) & (!is.na(gene_pairs$eGene.y))] = gene_pairs$gene2[(is.na(gene_pairs$eGene.x)) & (!is.na(gene_pairs$eGene.y))]
gene_pairs$eGene[(!is.na(gene_pairs$eGene.x)) & (!is.na(gene_pairs$eGene.y))] = gene_pairs$feature_id[(!is.na(gene_pairs$eGene.x)) & (!is.na(gene_pairs$eGene.y))]

In [116]:
### Get unique values and save

In [118]:
head(gene_pairs,2)

snp_id,gene2,gene1,feature_id,beta.x,p_value.x,assessed_allele.x,QTL.x,beta.y,p_value.y,assessed_allele.y,QTL.y,eGene.x,eGene.y,QTL_beta,QTL_p_value,eGene
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
10:100055816:G:A,CWF19L1,AAK1,AAK1_CWF19L1,NA,NA,NA,NA,-0.2145484,1.735811e-05,G,10:100055816:G:A:CWF19L1,NA,1,-0.2145484,1.735811e-05,CWF19L1
10:100055816:G:A,CWF19L1,ABCC1,ABCC1_CWF19L1,NA,NA,NA,NA,-0.2145484,1.735811e-05,G,10:100055816:G:A:CWF19L1,NA,1,-0.2145484,1.735811e-05,CWF19L1


In [119]:
gene_pair_co_eQTL_effects = unique(gene_pairs[,c('feature_id', 'snp_id', 'QTL_beta', 'QTL_p_value', 'eGene')])

In [120]:
nrow(gene_pair_co_eQTL_effects)

[1] 31497753

In [121]:
write.csv(gene_pair_co_eQTL_effects, paste0(results_path_analysis, 'F6_QTL_effects_summary.csv'))